In [65]:
import urllib2
import json
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap
from PIL import Image, ImageDraw
%matplotlib inline
import random

API_KEY = 'vjfSeIozbYBgOrnA9HJBuMRQF6arH8I2'
LEFT_BOTTOM= ['-73','41']
RIGHT_TOP = ['-68','46']

In [56]:
def get_box_hotel(left_bottom, right_top, limit=0):
    """Get hotel list according to their geo location(bounded box)
    
    left_bottom and right_top is a length 2 list 
    recording the longitude and lattitude of a bounded rectangular box.
    """
    if limit == 0:
        url = ''.join(('http://terminal2.expedia.com:80/x/geo/features?bbox=',
                       left_bottom[0], '%2C', left_bottom[1], '%2C', right_top[0], 
                       '%2C', right_top[1], '&type=hotel&apikey=', API_KEY))
    else:
        url = ''.join(('http://terminal2.expedia.com:80/x/geo/features?bbox=',
                       left_bottom[0], '%2C', left_bottom[1], '%2C', right_top[0], 
                       '%2C', right_top[1], '&type=hotel&limit=', limit,'&apikey=', API_KEY))
    data = json.load(urllib2.urlopen(url))
    
    return data



In [4]:
data = get_box_hotel(LEFT_BOTTOM, RIGHT_TOP)
print len(data)

10


In [48]:
im = Image.open("map.png")

left, right, up, down = 23, 470, 33, 303

draw = ImageDraw.Draw(im)
draw.line((left, 0, left, im.size[1]), fill=230)
draw.line((right, 0, right, im.size[1]), fill=230)
draw.line((0, up, im.size[0], up), fill=230)
draw.line((0, down, im.size[0], down), fill=230)

# plot grid
left_l, right_l, up_l, down_l = -73.7, -69.7, 43.1, 41
w_grad = (right_l - left_l)/(right - left)
h_grad = (up_l - down_l)/(down - up)
# grid deta lat, longti
deta_x, deta_y = 0.1/w_grad, 0.1/h_grad

for x in np.arange(left, right, deta_x):
    draw.line((x, 0, x, im.size[1]), fill=230)
for y in np.arange(up, down, deta_y):
    draw.line((0, y, im.size[0], y), fill=230)

im.show()

In [57]:
# Ground Truth, medium rectangle, upper small rectangle and right bottom rectangle
# whole MA, -73.7 ~ -69.7 W, 41 ~ 43.1 N
# count the grid 
rec1 = get_box_hotel([str(left_l), str(up_l-11*deta_y*h_grad)], 
                     [str(right_l), str(up_l-2*deta_y*h_grad)])
rec2 = get_box_hotel([str(left_l+25*deta_x*w_grad), str(up_l-2*deta_y*h_grad)], 
                     [str(left_l+33*deta_x*w_grad), str(up_l)])
rec3 = get_box_hotel([str(left_l+24*deta_x*w_grad), str(down_l)],
                     [str(right_l), str(up_l-11*deta_y*h_grad)])

len(rec1) + len(rec2) + len(rec3)

1215

In [75]:
# Sample method 1: random choose one point and grid size
all_box_count = 33*9 + 5*2 + 10*16 -14+3-31-21

x = random.choice(range(1, 40))
y = random.choice(range(1,21))
b = random.choice(range(5, 15))
print x, y, b

p = b*1.0 / all_box_count
sample1 = get_box_hotel([str(left_l+x*deta_x*w_grad), str(up_l-y*deta_y*h_grad)],
                        [str(left_l+(x+b)*deta_x*w_grad), str(up_l-(y-1)*deta_y*h_grad)])
len(sample1)

15 14 12


34

In [76]:
total = len(sample1)/p

In [77]:
total

1144.6666666666667